In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
search_images_ddg

Lets get bear images from duckduckgo_search

In [ ]:
ims = search_images_ddg('Grizzly bear')
len(ims)

call the image url and saving it to the dest path

In [ ]:
dest = 'images/grizzly.jpg'
download_url(ims[0],dest,show_progress=False)

calls and displays the image

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

Check/Create the root "bears" folder if it’s not there.

Loop over each bear type in bear_types.

Make a subfolder for that type.

Search Bing for images of “<type> bear” via search_images_bing.

Download all returned image URLs into the appropriate folder using download_images.

In [ ]:
if not path.exists():
  path.mkdir()
  for o in bear_types:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    results = search_images_ddg(f'{0} bear')
    download_images(dest,urls= results)


In [ ]:
fns = get_image_files(path)
fns

remove failed ones

In [ ]:
failed =verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

now we gathered the data, so its time to clean the dataa like beofre with data loader and data block
Blocks: Choose your input/output types (ImageBlock, CategoryBlock).

get_items: Gather all file paths.

get_y: Pull labels from those paths.

splitter: Split into train/validation sets.

(Optional) item_tfms: Preprocess each item (resize, crop, etc.).

dataloaders(): Materialize into actual DataLoaders ready for your model.

In [ ]:
bears = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items= get_image_files,
    splitter= RandomSplitter(valid_pct=0.2,seed = 42),
    get_y= parent_label,
    item_tfms= Resize(128)
)

They will go through our model in a big array called tensors, but they need to be same size, all of it.

In [ ]:
item_tfms= Resize(128)

This is like the template for dataloaders.

In [ ]:
dls = bears.dataloaders(path)

you gotta check the inputs, model hasnt been trained on.

In [ ]:
dls.valid.show_batch(max_n=4,nrows=1)

in traning we must make sure that all images are of the same size, because nuerall netwrokd espically cnns requoires every image in a batch to have exact same dimensions..
2:04
if not same size we coudlnt form a rectungalur tensor..
You need same sizes
2:06
You can do padding

In [ ]:
bears = bears.new(item_tfms=Resize(128,ResizeMethod.Squish))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4,nrows=1)

In [ ]:
bears = bears.new(item_tfms=Resize(128,ResizeMethod.Pad,pad_mode='zeros'))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4,nrows=1)

random reziedcrop is a very good method to crop image randomly, like the in every epoch, it will give model image from diffeent directioon, and its good for traning..

This is called data argumentation, getting different image from saem image everytime.

In [ ]:
bears = bears.new(item_tfms==RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4,nrows=1, unique = True)